In [74]:
# Class Vertice
class Vertex:
    '''Estrutura de Nó para um grafo: um elemento que é o identificador deste nó'''

    def __init__(self, x):
        '''O vértice será inserido no Grafo usando o método insert_vertex(x) que cria um Vertex'''
        self._elemento = x

    def __hash__(self):
        ''' este vértice (o seu identificador) é usado como chave'''
        return hash(id(self))  # devolve um inteiro que identifica este vértice como uma chave num dicionário

    def __str__(self):
        return'{0}'.format(self._elemento)

    def __eq__(self, x):
        return x == self._elemento

    def vertice(self):
        '''Devolve o nome deste vértice'''
        return self._elemento


# #### Class Edge
class Edge:
    '''Estrutura de Aresta para um Grafo: (origem, destino) e peso '''

    def __init__(self, u, v, p):
        self._ant = u
        self._suc = v
        self._weight = p

    def __hash__(self):
        # para associar a aresta a uma chave para um dicionário
        return hash( (self._ant, self._suc) )

    def __str__(self):
        '''Mostra a Aresta para um Grafo: (origem, destino) - peso '''
        return'({0},{1})-{2} '.format(self._ant, self._suc, self._weight)

    def __eq__(self, other):
        # define igualdade de duas arestas
        return self._ant == other._ant and self._suc == other._suc

    def endpoints(self):
        '''devolve (u,v) para indicar os vértices antecessor e sucessor.'''
        return (self._ant, self._suc)

    def opposite(self, v):
        '''Indica o vértice oposto ao v neste arco; v tem de ser um dos vértices.'''
        return self._suc if v is self._ant else self._ant

    def cost(self):
        '''Devolve o peso associado a este arco.'''
        return self._weight

    def show_edge(self):
        print('(',self._ant, ', ', self._suc, ') com peso', self._weight)

In [75]:
class Graph(Vertex, Edge):
    '''Representação de um grafo usando mapeamentos de adjacências (associações) - dictionaries'''

    def __init__(self, directed=False):
        '''Cria um grafo vazio (dicionário de _vertices); é orientado se o parâmetro directed tiver o valor True'''
        self._directed = directed
        self._n = 0                 # quantidade de nós no Grafo
        self._m = 0                 # quantidade de arcos no Grafo
        self._vertices = {}         # dicionário com chave vértice e valor dicionário de adjacências

    def insert_vertex(self, x):
        '''Insere e devolve um novo vértice com o elemento x'''
        v = Vertex(x)
        self._vertices[v] = {}      # inicializa o dicionário de adjacências deste vértice a vazio
        self._n +=1                 # mais um vértice no grafo
        return v

    def insert_edge(self, u, v, x):
        '''Cria e insere uma nova aresta entre u e v com peso x'''
        e = Edge(u, v, x)
        self._vertices[u][v] = e  # vai colocar nas adjacências de u
        self._vertices[v][u] = e  # e nas adjacências de v (para facilitar a procura de todos os arcos incidentes em ou originários de)
        self._m +=1

    def is_directed(self):
        '''com base na criação original da instância, devolve True se o Grafo é dirigido; False senão '''
        return self._directed  # True se os dois contentores são distintos

    def order(self):
        '''Ordem de um grafo: a quantidade de vértices no Grafo'''
        return self._n

    def vertices(self):
        '''Devolve um iterável sobre todos os vértices do Grafo'''
        return self._vertices.keys()

    def size(self):
        '''Dimensão de um grafo: a quantidade total de arestas do Grafo'''
        return self._m

    def set_of_edges(self):
        '''Devolve o conjunto (set) de todas as arestas do Grafo'''
        result = set()      # avoid double-reporting edges in undirected graph
        for secondary_map in self._vertices.values():
            result.update(secondary_map.values())  # add edges to resulting set
        return result


    def degree(self, v, outgoing=True):
        '''Quantidade de arestas originárias ou incidentes no vértice v
           Se for um grafo dirigido, conta as arestas outgoing ou incoming,
           de acordo com o valor de outgoing (True or False)
        '''
        adj = self._vertices
        if not self._directed:
            count = len(adj[v])
        else:
            count = 0
            for edge in adj[v].values():
                x, y = edge.endpoints()
                if (outgoing and x == v) or (not outgoing and y == v):
                    count += 1
        return count


    def get_edge(self, u, v):
        '''Método interno: Devolve a aresta que liga u a v ou None se não forem adjacentes'''
        edge = self._vertices[u].get(v)     # returns None se não existir aresta entre u e v
        if edge and self._directed: # se é dirigido
            x = edge.endpoints()        # vai confirmar se é o arco u --> v
            if x[1] != v:
                edge = None
        return edge

    def remove_edge(self, u, v):
        '''Remove a aresta entre u e v '''
        if  u in self._vertices.keys() and v in self._vertices[u].keys():
            del self._vertices[u][v]
            del self._vertices[v][u]
            self._m -=1

    def remove_vertex(self, v):
        '''remove o vértice v'''
        # remover todas as arestas de [v]
        # remover todas as arestas com v noutros vertices
        # remover o vértice v
        if v in self._vertices.keys():
            lst = [i for i in self.incident_edges(v)]
            for i in lst:
                x, y = i.endpoints()
                self.remove_edge(x,y)
            del self._vertices[v]
            self._n -=1
        #return v


    #outros métodos auxiliares#
    def incident_edges(self, v, incoming=True):
        '''Gerador: indica todas as arestas incoming de v
           Se for um grafo dirigido e incoming for False, devolve as arestas outgoing
        '''
        for edge in self._vertices[v].values(): # para todas as arestas relativas a v:
            if not self._directed:
                    yield edge
            else:  # senão deve ir procurar em todas as arestas para saber quais entram ou saiem
                x, y = edge.endpoints()
                if (incoming and y == v) or (not incoming and x == v):
                    yield edge

    def printG(self):
        '''Mostra o grafo por linhas'''
        if self._n == 0:
            print('O grafo está vazio!')
        else:
            print('Grafo orientado:', self._directed)
            for v in self.vertices():
                print('\nvertex ', v, ' grau_in: ', self.degree(v,False), end=' ')# mostra o grau (de entrada, se orientado)
                for i in self.incident_edges(v):
                    print(' ', i, end=' ')
                if self._directed:          # se orientado, mostrar o grau de saída
                    print('\n \t   grau_out: ', self.degree(v, True), end=' ')
                    for i in self.incident_edges(v, False):    # e mostra os arcos de saída
                        print(' ', i, end=' ')

Questão 1.- Algoritmo de Dijkstra (com estrutura de dados auxiliar uma Heap binária)

In [76]:
"""
Aqui está uma abordagem geral para encontrar os k caminhos mais curtos usando uma variação do algoritmo de Dijkstra:

Inicialize uma estrutura de dados para armazenar os k caminhos mais curtos. Isso pode ser uma lista de caminhos, onde cada caminho é uma sequência de vértices.
Inicialize todas as distâncias dos vértices como infinito, exceto a distância do vértice de origem, que é definida como 0.
Crie uma fila de prioridade (Heap binária, por exemplo) para armazenar os vértices a serem processados, ordenados pela distância.
Insira o vértice de origem na fila de prioridade com distância 0.
Enquanto a fila de prioridade não estiver vazia e o número de caminhos encontrados for menor que k, faça o seguinte:
Remova o vértice com a menor distância da fila de prioridade (vértice atual).
Se o vértice atual for o destino de um caminho completo (ou seja, já foi encontrado um caminho até ele), ignore-o e continue para o próximo vértice.
Para cada vértice adjacente ao vértice atual, faça o seguinte:
Calcule a distância até o vértice adjacente através do vértice atual.
Se a nova distância for menor do que a distância atual do vértice adjacente, atualize a distância do vértice adjacente.
Atualize ou adicione o caminho para o vértice adjacente na estrutura de dados de caminhos, mantendo apenas os k caminhos mais curtos.
Insira o vértice adjacente na fila de prioridade com a nova distância.
Após processar todos os vértices ou encontrar k caminhos mais curtos, você terá os k caminhos mais curtos do vértice de origem para os outros vértices armazenados na estrutura de dados de caminhos.
"""
import heapq
from collections import defaultdict

class MinHeapItem:
    def __init__(self, priority, value):
        self.priority = priority
        self.value = value

    def __lt__(self, other):
        return self.priority < other.priority

def dijkstra_k_shortest_paths(graph, source, k):
    distances = {}
    previous = {}
    pq = []

    for vertex in graph.vertices():
        distances[vertex] = float('inf')
        previous[vertex] = None

    distances[source] = 0
    heapq.heappush(pq, MinHeapItem(distances[source], source))

    while pq:
        current_item = heapq.heappop(pq)
        current_dist = current_item.priority
        current_vertex = current_item.value

        if current_dist > distances[current_vertex]:
            continue

        for edge in graph.incident_edges(current_vertex):
            neighbor = edge.opposite(current_vertex)
            weight = edge.cost()
            new_dist = distances[current_vertex] + weight

            if new_dist < distances[neighbor]:
                distances[neighbor] = new_dist
                previous[neighbor] = current_vertex
                heapq.heappush(pq, MinHeapItem(new_dist, neighbor))

    paths = {}
    for vertex in graph.vertices():
        if vertex != source:
            paths[vertex] = []
            vertex_prev = vertex
            for _ in range(k):
                if vertex_prev is None:
                    break
                paths[vertex].append(vertex_prev)
                vertex_prev = previous[vertex_prev]
            paths[vertex].reverse()

    return distances, paths

In [87]:
# Criação do grafo de exemplo
graph = Graph()
v1 = graph.insert_vertex('A')
v2 = graph.insert_vertex('B')
v3 = graph.insert_vertex('C')
v4 = graph.insert_vertex('D')
v5 = graph.insert_vertex('E')
graph.insert_edge(v1, v2, 1)
graph.insert_edge(v1, v3, 4)
graph.insert_edge(v2, v3, 2)
graph.insert_edge(v2, v4, 5)
graph.insert_edge(v3, v4, 1)
graph.insert_edge(v3, v5, 3)
graph.insert_edge(v4, v5, 2)

# Execução do algoritmo para encontrar os 3 caminhos mais curtos a partir do vértice 'A'
source = v1
k = 3
distances, paths = dijkstra_k_shortest_paths(graph, source, k)

# Exibição dos resultados
print("Distâncias mínimas:")
for vertex, distance in distances.items():
    print(f"{vertex}: {distance}")

print("\nCaminhos mais curtos:")
for vertex, path in paths.items():
    path_vertices = [str(v) for v in path]  # Converte os objetos Vertex em strings
    print(f"{vertex}: {' -> '.join(path_vertices)}")

Distâncias mínimas:
A: 0
B: 1
C: 3
D: 4
E: 6

Caminhos mais curtos:
B: A -> B
C: A -> B -> C
D: B -> C -> D
E: B -> C -> E


Questão 2- Algoritmo de Yen para a determinação dos k caminhos mais curtos entre
dois dados nós num grafo

In [84]:
##não está a funcionar

def yen_k_shortest_paths(Graph, source, target, k):
    # Dijkstra para encontrar o caminho mais curto inicial
    distances, paths = dijkstra_k_shortest_paths(Graph, source, 1)
    shortest_path = paths[target]

    k_shortest_paths = [shortest_path]  # lista para armazenar os k caminhos mais curtos

    for i in range(1, k):
        for j in range(len(shortest_path) - 1):
            spur_node = shortest_path[j]
            root_path = shortest_path[:j + 1]

            # Remover arestas do grafo
            Graph.remove_edge(root_path[-1], shortest_path[j + 1])

            # Remover vértices que não estão no caminho mais curto atual
            for path in k_shortest_paths:
                if len(path) > j and root_path == path[:j + 1]:
                    Graph.remove_vertex(path[j + 1])

            # Encontrar o spur path usando Dijkstra
            spur_distances, spur_paths = dijkstra_k_shortest_paths(Graph, spur_node, target, 1)

            if target in spur_paths:
                spur_path = spur_paths[target]
                total_path = root_path[:-1] + spur_path

                # Adicionar caminho ao conjunto de caminhos mais curtos
                if total_path not in k_shortest_paths:
                    k_shortest_paths.append(total_path)

            # Restaurar grafo original
            Graph.insert_edge(root_path[-1], shortest_path[j + 1], distances[(root_path[-1], shortest_path[j + 1])])

            for path in k_shortest_paths:
                if len(path) > j and root_path == path[:j + 1]:
                    Graph.insert_vertex(path[j + 1])

        if len(k_shortest_paths) < i + 1:
            # Não existem mais caminhos disponíveis
            break

        # Ordenar os caminhos mais curtos de acordo com a distância total
        k_shortest_paths.sort(key=lambda path: sum(Graph.get_edge(path[j], path[j + 1]).cost() for j in range(len(path) - 1)))

        # Atualizar o caminho mais curto
        shortest_path = k_shortest_paths[i]

    return k_shortest_paths


In [83]:
# Criar um grafo
grafo = Graph(directed=True)

# Inserir vértices
A = grafo.insert_vertex('A')
B = grafo.insert_vertex('B')
C = grafo.insert_vertex('C')
D = grafo.insert_vertex('D')
E = grafo.insert_vertex('E')

# Inserir arestas
grafo.insert_edge(A, B, 1)
grafo.insert_edge(A, C, 5)
grafo.insert_edge(B, C, 2)
grafo.insert_edge(C, D, 3)
grafo.insert_edge(D, E, 4)

# Encontrar os 3 caminhos mais curtos entre A e E
caminhos = yen_k_shortest_paths(grafo, A, E, 3)
print(caminhos)
# Exibir os caminhos
for i, caminho in enumerate(caminhos, start=1):
    caminho_vertices = [str(v) for v in caminho]  # Converte os objetos Vertex em strings
    print(f'Caminho {i}: {caminho_vertices}')
    total_distance = sum(grafo.get_edge(caminho[j], caminho[j + 1]).cost() for j in range(len(caminho) - 1))
    print(f'Distância total: {total_distance}')
    print('---')

[[<__main__.Vertex object at 0x000002627CFBC070>]]
Caminho 1: ['E']
Distância total: 0
---
